<font size="6"><b>S16 T02: Tasca de web scraping</b></font>

# Enunciats
**Nivell 1**
1. Realitza web scraping d'una pàgina de la borsa de Madrid (https://www.bolsamadrid.es) utilitzant BeautifulSoup i Selenium.


**Nivell 2**  

2. Documenta en un word el teu conjunt de dades generat amb la informació que tenen els diferents arxius de Kaggle.


**Nivell 3**  

3. Tria una página web que tu vulguis i realitza web scraping mitjançant la llibreria Scrapy.

## Web scraping la Borsa de Madrid

### amb BeautifulSoup.

In [6]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

url = 'https://www.bolsamadrid.es/esp/aspx/Portada/Portada.aspx'
page = urlopen(url)
soup = BeautifulSoup(page,"html.parser")
#print(soup.prettify())

In [7]:
for link in soup.find_all('a'):
    if 'href' in link.attrs:
        links=(link.attrs['href'])

In [27]:
for link in soup.find('div', {'id':'Base'}).find_all('a', string='Acciones'):
    if 'href' in link.attrs:
        tails =link.attrs['href']
        #print(tails)[:1]
        print(link.attrs['href']) 

/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000
/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000


In [35]:
tails = [link.attrs['href'] for link in soup.find('div', {'id':'Base'}).find_all(
    'a', string='Acciones') if 'href' in link.attrs]


/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000


In [36]:
tail = tails[-1]
tail

'/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000'

In [37]:
url = 'https://www.bolsamadrid.es'
newUrl = url+tails[-1]
newUrl

'https://www.bolsamadrid.es/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000'

In [49]:
newpage = urlopen(newUrl)
soup = BeautifulSoup(newpage,"html.parser")


In [50]:
accions = []
acc_tables = soup.find_all('table', class_="TblPort")

for table in acc_tables:
    headers =[]
    rows = table.find_all('tr')
    #start by fetching the header cells from the first row to determine the field names
    for header in table.find('tr').find_all('th'):
        headers.append(header.text)
    #then go through all the rows except the first one
    for row in table.find_all('tr')[1:]:
        values = []
        #And get the column cells, the first one being inside a th-tag
        for col in row.find_all(['th', 'td']):
            values.append(col.text)
        if values:
            acc_dict = {headers[i]:values[i] for i in range(len(values))}
            accions.append(acc_dict)


In [52]:
accions[:2] 

[{'Nombre': 'IBEX 35®',
  'Anterior': '8.991,50',
  'Último': '8.988,20',
  '% Dif.': '-0,04',
  'Máximo': '9.020,70',
  'Mínimo': '8.972,10',
  'Fecha': '02/09/2021',
  'Hora': '13:00:23',
  '% Dif.Año 2021': '11,33'},
 {'Nombre': 'ACCIONA',
  'Últ.': '141,2000',
  '% Dif.': '1,00',
  'Máx.': '141,7000',
  'Mín.': '139,5000',
  'Volumen': '14.787',
  'Efectivo (miles €)': '2.083,61',
  'Fecha': '02/09/2021',
  'Hora': '12:59:09'}]

In [46]:
del accions[0]

In [47]:
import pandas as pd 

df = pd.DataFrame(accions)
df.head()

,Nombre,Últ.,% Dif.,Máx.,Mín.,Volumen,Efectivo (miles €),Fecha,Hora
0,ACCIONA,"141,1000","0,93","141,7000","139,5000",12.954,"1.824,78",02/09/2021,12:27:08
1,ACERINOX,"11,4550","-0,52","11,5750","11,4400",174.838,"2.012,16",02/09/2021,12:33:27
2,ACS,"23,5900","0,34","23,6400","23,4000",111.737,"2.631,38",02/09/2021,12:30:47
3,AENA,"137,8000","-0,22","139,2500","136,9500",11.588,"1.601,74",02/09/2021,12:33:00
4,ALMIRALL,"14,7300","0,68","14,7500","14,4500",115.102,"1.687,89",02/09/2021,12:31:06


### amb Selenium.

In [8]:
from selenium import webdriver
import pandas as pd 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

url = 'https://www.bolsamadrid.es'

In [9]:
driver =webdriver.Chrome()
driver.get(url)
WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="Base"]/table/tbody/tr[4]/td[2]/div[1]/div[2]/table/tbody/tr[1]/td[1]/a'))).click()

Així obtenim les dades de la pàgina, però les volem tenir en un DataFrame per poder treballar amb elles...

In [10]:
taula = driver.find_elements(By.XPATH,'//*[@id="ctl00_Contenido_tblAcciones"]')
for i in taula:
    print(i.text)


Nombre Últ. % Dif. Máx. Mín. Volumen Efectivo (miles €) Fecha Hora
ACCIONA 141,5000 -0,07 142,0000 141,0000 12.974 1.835,60 06/09/2021 12:11:11
ACERINOX 11,3300 -0,79 11,5000 11,3100 141.313 1.609,19 06/09/2021 12:15:48
ACS 23,4500 0,77 23,4700 23,2300 110.994 2.588,40 06/09/2021 12:16:51
AENA 134,1500 0,19 134,7000 133,4000 14.545 1.948,37 06/09/2021 12:15:45
ALMIRALL 14,6300 0,48 14,6600 14,5300 24.439 356,90 06/09/2021 12:15:18
AMADEUS 51,5600 0,31 51,9600 51,3400 202.778 10.458,44 06/09/2021 12:16:01
ARCELORMIT. 28,6050 1,27 28,6050 28,2800 78.322 2.231,65 06/09/2021 12:15:56
B.SANTANDER 3,1115 1,15 3,1255 3,0840 3.837.596 11.920,04 06/09/2021 12:16:10
BA.SABADELL 0,5996 0,17 0,6052 0,5964 3.640.852 2.186,21 06/09/2021 12:16:14
BANKINTER 4,9610 0,43 4,9870 4,9440 212.497 1.055,60 06/09/2021 12:15:44
BBVA 5,5780 0,18 5,6360 5,5650 1.239.794 6.935,32 06/09/2021 12:16:23
CAIXABANK 2,6530 0,38 2,6650 2,6380 790.943 2.097,89 06/09/2021 12:15:59
CELLNEX 58,4200 1,56 58,7600 58,0000 133.3

ara n'extreurem les dades, dels titols, i els convinarem en un DataFrame.

In [11]:
headers = [header.text for header in driver.find_elements(By.XPATH,'//*[@id="ctl00_Contenido_tblAcciones"]/tbody/tr/th')]


In [12]:
dades = [dada.text for dada in driver.find_elements(By.XPATH,'//*[@id="ctl00_Contenido_tblAcciones"]/tbody/tr/td')] 


In [13]:
#dividirem les dades en llistes q continguin la informació de cada companyia
#mida dels grups, ha de ser ingual de llarg q el grup del titols
n = len(headers)
  
# ho farem amb una comprehension list
dades_final = [dades[i * n:(i + 1) * n] for i in range((len(dades) + n - 1) // n )] 
 

In [14]:
import pandas as pd 

df = pd.DataFrame(dades_final, columns=headers)
df.head()



,Nombre,Últ.,% Dif.,Máx.,Mín.,Volumen,Efectivo (miles €),Fecha,Hora
0,ACCIONA,"141,5000","-0,07","142,0000","141,0000",12.974,"1.835,60",06/09/2021,12:11:11
1,ACERINOX,"11,3300","-0,79","11,5000","11,3100",141.313,"1.609,19",06/09/2021,12:15:48
2,ACS,"23,4500","0,77","23,4700","23,2300",110.994,"2.588,40",06/09/2021,12:16:51
3,AENA,"134,1500","0,19","134,7000","133,4000",14.545,"1.948,37",06/09/2021,12:15:45
4,ALMIRALL,"14,6300","0,48","14,6600","14,5300",24.439,"356,90",06/09/2021,12:15:18


Ja ho tenim llest per treballar amb les dades, exportar-ho a un document, etc.

In [15]:
driver.quit()

### Amb selenium fent un filtre previ

In [16]:
from selenium import webdriver
import pandas as pd 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

url = 'https://www.bolsamadrid.es'

In [17]:
driver =webdriver.Chrome()
driver.get(url)
WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="Base"]/table/tbody/tr[4]/td[2]/div[1]/div[2]/table/tbody/tr[1]/td[1]/a'))).click()

Si vulguessim fer servir el filtre d'inici de la pàgina, afegririem el següent:

In [18]:
from selenium.webdriver.support.ui import Select
select = Select(driver.find_element_by_xpath('//*[@id="SelSector"]'));
select.select_by_visible_text("Tecnología y Telecomunicaciones");
driver.find_element_by_xpath('//*[@id="ctl00_Contenido_Consultar"]').click() 
WebDriverWait(driver, 20)

<selenium.webdriver.support.wait.WebDriverWait (session="7d8d2e3a2c5c23e5bb6cb65a77b3aa48")>

In [19]:
taula = driver.find_elements(By.XPATH,'//*[@id="ctl00_Contenido_tblAcciones"]')
for i in taula:
    print(i.text)


Nombre Mercado Últ. % Dif. Máx. Mín. Volumen Efectivo (miles €) Fecha Hora
AETERNAL M. MAB - EE 8,9500 0,00 8,9500 8,9500 2.674 23,93 06/09/2021 12:00:06
AGILE MAB - EE 7,6000 1,33 7,6000 7,5000 3.140 23,65 06/09/2021 11:56:39
ALTIA MAB - EE 27,2000 -2,16 27,2000 27,2000 25 0,68 06/09/2021 12:00:06
AMADEUS M.Continuo 51,5800 0,35 51,9600 51,3400 202.947 10.467,16 06/09/2021 12:17:48
AMERICA M. L Latibex 0,8050 - - - - - 03/09/2021 Cierre
AMPER M.Continuo 0,1838 0,22 0,1890 0,1824 574.002 107,09 06/09/2021 11:48:19
CATENON MAB - EE 1,1000 0,46 1,1000 1,0800 3.050 3,32 06/09/2021 11:37:10
CELLNEX M.Continuo 58,3800 1,50 58,7600 58,0000 133.821 7.818,83 06/09/2021 12:17:21
CLEVER MAB - EE 0,2340 - - - - - 03/09/2021 Cierre
CUATROOCHENT MAB - EE 17,0000 0,00 17,0000 17,0000 100 1,70 06/09/2021 09:00:09
DOMINION M.Continuo 4,5300 0,67 4,5300 4,4950 63.432 286,45 06/09/2021 12:06:00
EURONA MAB - EE 0,1960 - - - - - Suspendido 30/04/2020
EZENTIS M.Continuo 0,3540 0,00 0,3560 0,3510 145.864 51

ara n'extreurem les dades, dels titols, i els convinarem en un DataFrame.

In [20]:
headers = [header.text for header in driver.find_elements(By.XPATH,'//*[@id="ctl00_Contenido_tblAcciones"]/tbody/tr/th')]
dades = [dada.text for dada in driver.find_elements(By.XPATH,'//*[@id="ctl00_Contenido_tblAcciones"]/tbody/tr/td')] 

In [21]:
#dividirem les dades en llistes q continguin la informació de cada companyia
#mida dels grups, ha de ser ingual de llarg q el grup del titols
n = len(headers)
  
# ho farem amb una comprehension list
dades_final = [dades[i * n:(i + 1) * n] for i in range((len(dades) + n - 1) // n )] 
 

In [22]:
import pandas as pd 

df = pd.DataFrame(dades_final, columns=headers)
df.head()



,Nombre,Mercado,Últ.,% Dif.,Máx.,Mín.,Volumen,Efectivo (miles €),Fecha,Hora
0,AETERNAL M.,MAB - EE,"8,9500","0,00","8,9500","8,9500",2.674,"23,93",06/09/2021,12:00:06
1,AGILE,MAB - EE,"7,6000","1,33","7,6000","7,5000",3.140,"23,65",06/09/2021,11:56:39
2,ALTIA,MAB - EE,"27,2000","-2,16","27,2000","27,2000",25,"0,68",06/09/2021,12:00:06
3,AMADEUS,M.Continuo,"51,5800","0,35","51,9600","51,3400",202.947,"10.467,16",06/09/2021,12:17:48
4,AMERICA M. L,Latibex,"0,8050",-,-,-,-,-,03/09/2021,Cierre


In [23]:
driver.quit()

## Web scraping la Borsa de Madrid-Documentació

arxiu txt que podeu trobar en aquest link [Dataset Documentation](https://raw.githubusercontent.com/Ortegbel/PractiquesDataScience/master/S16/S06_T02%20Dataset%20Documentation)



## Scrapy



In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess

In [2]:
import json
class JsonWriterPipeline(object):
    
    def open_spider(self, spider):
        self.file = open('quoteresult.jl','w')
        
    def close_spider(self, spider):
        self.file.close()
        
    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [3]:
import logging

class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        'http://quotes.toscrape.com/page/1/',
        'http://quotes.toscrape.com/page/2/',
    ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FORMAT':'json',                                 # Used for pipeline 2
        'URI': 'quoteresult.json'                        # Used for pipeline 2
    }
    
    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').extract_first(),
                'author': quote.css('span small::text').extract_first(),
                'tags': quote.css('div.tags a.tag::text').extract(),
            }

In [4]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(QuotesSpider)
process.start()

2021-09-06 12:14:07 [scrapy.utils.log] INFO: Scrapy 2.5.0 started (bot: scrapybot)
2021-09-06 12:14:07 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.7.0, Python 3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 20.0.1 (OpenSSL 1.1.1k  25 Mar 2021), cryptography 3.4.7, Platform Windows-10-10.0.19041-SP0
2021-09-06 12:14:07 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-09-06 12:14:07 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 30,
 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}


In [5]:
import pandas as pd 
dfjl = pd.read_json('quoteresult.jl',lines=True)
dfjl

,text,author,tags
0,“The world as we have created it is a process ...,Albert Einstein,"[change, deep-thoughts, thinking, world]"
1,“This life is what you make it. No matter what...,Marilyn Monroe,"[friends, heartbreak, inspirational, life, lov..."
2,“It takes a great deal of bravery to stand up ...,J.K. Rowling,"[courage, friends]"
3,"“If you can't explain it to a six year old, yo...",Albert Einstein,"[simplicity, understand]"
4,"“You may not be her first, her last, or her on...",Bob Marley,[love]
5,"“I like nonsense, it wakes up the brain cells....",Dr. Seuss,[fantasy]
6,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"[abilities, choices]"
7,“There are only two ways to live your life. On...,Albert Einstein,"[inspirational, life, live, miracle, miracles]"
8,"“The person, be it gentleman or lady, who has ...",Jane Austen,"[aliteracy, books, classic, humor]"
9,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"[be-yourself, inspirational]"
